# **Application of ML Algorithms**

## **Data Preparation and Environment Setup**

Different Machiene Learning Classification Algorithms were applied on the dataset.

This is divided into 2 notebooks and this is the first one.

The classification Algorithms applied and compared in this notebook are 

1.   Logistic Regression
2.   Logistic Regression with SMOTE Resampling
3.   Decision Tree Classifier
4.   Random Forest Classifier
5.   Gradient Boosting Classifier
6.   CAT Boost Classifier

As the data is highly skewed, all the above models are tried with and without resampling an the results are analysed.

Second round of data preparation was essential and number of classes were reduced. 

Google's Colab has free GPUs and users can run 2 sessions at a time.

This notebook was run on colab with Python 3 and GPU with 25Gi RAM.


In [0]:
from google.colab import auth

In [0]:
!pip install gcloud

     |████████████████████████████████| 460kB 4.9MB/s 
  Created wheel for gcloud: filename=gcloud-0.18.3-cp36-none-any.whl size=602939 sha256=ca8751ca972917922697c55dfbf52e84f7f947d230e77e08475173d369bf706a
  Stored in directory: /root/.cache/pip/wheels/b9/9b/9c/a01be401658fea33b93a35d03921b0c638266821b264dc8662
Successfully built gcloud


Authenticate yourself to colab

In [0]:
auth.authenticate_user()

In [0]:
!pip install gcsfs

Set the project on GCP

In [0]:
!gcloud config set project skilful-orb-255314

Updated property [core/project].


Copy the three_class_model_train.csv file which was prepared in data-prep-notebook-2 to the Disk.

In [0]:
!gsutil cp gs://dataproc-e3bd1f7b-2e29-4da6-a5c4-077c164fd32a-us-central1/avito/test/three_class_model_train.csv /train.csv

Copying gs://dataproc-e3bd1f7b-2e29-4da6-a5c4-077c164fd32a-us-central1/avito/test/three_class_model_train.csv...
\ [1 files][137.4 MiB/137.4 MiB]                                                
Operation completed over 1 objects/137.4 MiB.                                    


**HACK For More RAM**

Colab by default gives 12.5 Gi of RAM an there are no settings to configure and increase it.

Only way to get 25 Gi of RAM is to crash the container by running out of Memory.

After crashing, memory can be upgraded. 

Lets Crash it,

In [0]:
arr=[]
while(1):
  arr.append('42')

Import libs

In [0]:
import pandas as pd
import numpy as np
import gcsfs

In [0]:
df_train = pd.read_csv('/train.csv')
df_train.head()

,region_en,category_name_en,parent_category_name_en,user_type,weekend,price,description_len,title_len,param_combined_len,image_present,image_top_1,deal_class_5
0,Sverdlovsk oblast,Children's products and toys,Personal belongings,Private,0,400.0,7,3,2,False,1008.0,Poor
1,Samara oblast,Furniture and interior,For the home and garden,Private,1,3000.0,7,3,1,False,692.0,Poor
2,Rostov oblast,Audio and video,Consumer electronics,Private,1,4000.0,17,2,5,False,3032.0,Okay
3,Tatarstan,Children's products and toys,Personal belongings,Company,0,2200.0,3,1,2,False,796.0,Good
4,Volgograd oblast,Cars,Transport,Private,0,40000.0,4,3,5,False,2264.0,Poor


These columns are selected after data preparation:


1.   region_en : Region where the ad was posted
2.   category_name_en : Category name of the product. 
3.   parent_category_name_en : Parent Category name of the product. 
4.   user_type : Type of user
5.   weekend : Whether the ad was posted on weekday or weekend.
6.   price : Selling price of the product.
7.   description_len : As NLP and tokenizer will be used in the later notebooks, just the number of words in description is taken into acount.
8.   title_len : Just like description, number of words in title is considered.
9.   param_combined_len : All the 3 optional parameters are combined and the total number of words in parameters combined is considered.
10.  image_present : Boolean column with True if image present else false.
11.  image_top_1 : Avito's code for image quality.

**Note** : category_name_en was eliminated and only parent_category_name_en was considered for modelling due to the large number of categorical vaiables present.



Target variable **deal_class_5** consists on 3 classes :


1.   Poor [Deal Probability between 0 and 0.3]
2.   Okay [Deal Probability between 0.3 and 0.7]
3.   Good [Deal Probability between 0.7 and 1]

These classes were fixed after multiple rounds of selection of number of classes and class intervals. The selected one did the best when Classification alorithms were applied.



In [0]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503424 entries, 0 to 1503423
Data columns (total 12 columns):
region_en                  1503424 non-null object
category_name_en           1503424 non-null object
parent_category_name_en    1503424 non-null object
user_type                  1503424 non-null object
weekend                    1503424 non-null int64
price                      1503424 non-null float64
description_len            1503424 non-null int64
title_len                  1503424 non-null int64
param_combined_len         1503424 non-null int64
image_present              1503424 non-null bool
image_top_1                1503424 non-null float64
deal_class_5               1503424 non-null object
dtypes: bool(1), float64(2), int64(4), object(5)
memory usage: 127.6+ MB


Import Libraries :

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix 
import time

Dependent Variable X and Independent Variable y

In [0]:
X = df_train[['region_en','parent_category_name_en','user_type','weekend','price','description_len','title_len','param_combined_len','image_present','image_top_1']]
y = df_train[['deal_class_5']]

Encoding is done on 3 categorical variables 

*   region_en
*   parent_category_name_en
*   user_type




In [0]:
X_enc = pd.get_dummies(X, columns=['region_en','user_type','parent_category_name_en'], drop_first = True)
X_enc.head()

,weekend,price,description_len,title_len,param_combined_len,image_present,image_top_1,region_en_Bashkortostan,region_en_Belgorod oblast,region_en_Chelyabinsk oblast,region_en_Irkutsk oblast,region_en_Kaliningrad oblast,region_en_Kemerovo oblast,region_en_Khanty-Mansi Autonomous Okrug,region_en_Krasnodar Krai,region_en_Krasnoyarsk Krai,region_en_Nizhny Novgorod oblast,region_en_Novosibirsk oblast,region_en_Omsk oblast,region_en_Orenburg oblast,region_en_Perm Krai,region_en_Rostov oblast,region_en_Samara oblast,region_en_Saratov oblast,region_en_Stavropol Krai,region_en_Sverdlovsk oblast,region_en_Tatarstan,region_en_Tula oblast,region_en_Tyumen oblast,region_en_Udmurtia,region_en_Vladimir oblast,region_en_Volgograd oblast,region_en_Voronezh oblast,region_en_Yaroslavl oblast,user_type_Private,user_type_Shop,parent_category_name_en_Consumer electronics,parent_category_name_en_For business,parent_category_name_en_For the home and garden,parent_category_name_en_Hobbies & leisure,parent_category_name_en_Personal belongings,parent_category_name_en_Real estate,parent_category_name_en_Services,parent_category_name_en_Transport
0,0,400.0,7,3,2,False,1008.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
1,1,3000.0,7,3,1,False,692.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
2,1,4000.0,17,2,5,False,3032.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
3,0,2200.0,3,1,2,False,796.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,40000.0,4,3,5,False,2264.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1


Test Train split : 12% of observations used for test.




In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_enc, y, test_size = 0.12, random_state = 0, stratify=y)

Validation Train split : 12% of observations from the previously train set used for validation.



In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.12, random_state = 0, stratify=y_train)

Normaliation of numerical columns

In [0]:
sc = StandardScaler()
X_train.loc[:,["price","description_len","title_len","param_combined_len","image_top_1"]] = sc.fit_transform(X_train[["price","description_len","title_len","param_combined_len","image_top_1"]])
X_test.loc[:,["price","description_len","title_len","param_combined_len","image_top_1"]] = sc.transform(X_test[["price","description_len","title_len","param_combined_len","image_top_1"]])
X_valid.loc[:,["price","description_len","title_len","param_combined_len","image_top_1"]] = sc.transform(X_valid[["price","description_len","title_len","param_combined_len","image_top_1"]])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s




---



## **Modelling**

As the data is highy skewed and **Okay : Good : Poor** are in the ratio
**1 : 2 : 15** ,

**F1 score** is considered as evaluation metric.

### Logistic Regression

**Class weigth given as balanced**

In [0]:
clf = LogisticRegression(class_weight="balanced")
# Fit the model on the trainng data.
clf.fit(X_train, y_train.values.ravel())

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Validation Set : 

In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.17      0.39      0.24     17217
        Okay       0.15      0.62      0.25      8730
        Poor       0.92      0.59      0.72    132815

    accuracy                           0.57    158762
   macro avg       0.42      0.53      0.40    158762
weighted avg       0.80      0.57      0.64    158762

[[ 6642  5257  5318]
 [ 2252  5412  1066]
 [29850 24429 78536]]


 **Sampling using SMOTE**

In [0]:
!pip install imblearn

In [0]:
  from imblearn.over_sampling import SMOTE
  import time

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
sm = SMOTE(sampling_strategy='auto',random_state=27)
y_train_smote = y_train.iloc[:, 0]
start = time.time()
X_train_smote, y_train_smote = sm.fit_sample(X_train, y_train_smote)
smote = LogisticRegression().fit(X_train_smote, y_train_smote)
end = time.time()
print("Time in seconds : ", end - start)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Time in seconds :  265.06606698036194


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Validation Set :

In [0]:
predictions = smote.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.17      0.39      0.24     17217
        Okay       0.15      0.62      0.25      8730
        Poor       0.92      0.59      0.72    132815

    accuracy                           0.57    158762
   macro avg       0.42      0.53      0.40    158762
weighted avg       0.80      0.57      0.64    158762

[[ 6646  5239  5332]
 [ 2247  5412  1071]
 [29836 24276 78703]]


Logistic Regression Prediction on Test Set :

In [0]:
predictions = smote.predict(X_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

        Good       0.17      0.38      0.23     19565
        Okay       0.15      0.62      0.25      9920
        Poor       0.93      0.59      0.72    150926

    accuracy                           0.57    180411
   macro avg       0.42      0.53      0.40    180411
weighted avg       0.80      0.57      0.64    180411

[[ 7397  6171  5997]
 [ 2621  6112  1187]
 [33879 27556 89491]]




---



### Decision Trees

In [0]:
from sklearn.tree import DecisionTreeClassifier

Simple Decision Tree with balanced weight :

In [0]:
clf = DecisionTreeClassifier(class_weight="balanced")
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)


In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.24      0.26      0.25     17217
        Okay       0.15      0.18      0.17      8730
        Poor       0.88      0.86      0.87    132815

    accuracy                           0.75    158762
   macro avg       0.42      0.43      0.43    158762
weighted avg       0.77      0.75      0.76    158762

[[  4548   1905  10764]
 [  1828   1555   5347]
 [ 12574   6648 113593]]


Use Entropy as criterion and set max depth 10:



In [0]:
clf = DecisionTreeClassifier(criterion="entropy", max_depth = 10,class_weight="balanced")
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)


In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.22      0.50      0.30     17217
        Okay       0.16      0.60      0.25      8730
        Poor       0.94      0.61      0.74    132815

    accuracy                           0.60    158762
   macro avg       0.44      0.57      0.43    158762
weighted avg       0.82      0.60      0.67    158762

[[ 8637  4799  3781]
 [ 2451  5233  1046]
 [28433 23599 80783]]


Use Gini Index as criterion and set max depth 10:

In [0]:
clf = DecisionTreeClassifier(criterion="gini", max_depth = 10,class_weight="balanced")
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.21      0.54      0.30     17217
        Okay       0.16      0.58      0.25      8730
        Poor       0.95      0.58      0.72    132815

    accuracy                           0.58    158762
   macro avg       0.44      0.57      0.42    158762
weighted avg       0.82      0.58      0.65    158762

[[ 9285  4488  3444]
 [ 2648  5093   989]
 [33045 22149 77621]]


Use Entropy as criterion and set max depth 15:

In [0]:
clf = DecisionTreeClassifier(criterion="entropy", max_depth = 15,class_weight="balanced")
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)


In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.22      0.53      0.31     17217
        Okay       0.16      0.56      0.25      8730
        Poor       0.94      0.62      0.75    132815

    accuracy                           0.61    158762
   macro avg       0.44      0.57      0.44    158762
weighted avg       0.82      0.61      0.67    158762

[[ 9138  4344  3735]
 [ 2571  4913  1246]
 [29306 21288 82221]]


Use Gini Index as criterion and set max depth 15:

In [0]:
clf = DecisionTreeClassifier(criterion="gini", max_depth = 15,class_weight="balanced")
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)


In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.23      0.52      0.32     17217
        Okay       0.16      0.57      0.25      8730
        Poor       0.94      0.62      0.75    132815

    accuracy                           0.61    158762
   macro avg       0.44      0.57      0.44    158762
weighted avg       0.82      0.61      0.67    158762

[[ 9017  4448  3752]
 [ 2532  4944  1254]
 [28345 22064 82406]]


Use Entropy as criterion and set max depth 7:

In [0]:
clf = DecisionTreeClassifier(criterion="entropy", max_depth = 7,class_weight="balanced")
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)


In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.20      0.38      0.27     17217
        Okay       0.13      0.71      0.22      8730
        Poor       0.94      0.56      0.71    132815

    accuracy                           0.55    158762
   macro avg       0.43      0.55      0.40    158762
weighted avg       0.82      0.55      0.63    158762

[[ 6593  7068  3556]
 [ 1668  6178   884]
 [24037 33882 74896]]


Use Entropy as Gini Index and set max depth 7:

In [0]:
clf = DecisionTreeClassifier(criterion="gini", max_depth = 7,class_weight="balanced")
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)


In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.20      0.50      0.28     17217
        Okay       0.14      0.62      0.23      8730
        Poor       0.95      0.55      0.70    132815

    accuracy                           0.55    158762
   macro avg       0.43      0.56      0.40    158762
weighted avg       0.82      0.55      0.63    158762

[[ 8597  5228  3392]
 [ 2473  5405   852]
 [32192 27210 73413]]


Without depth with Smote resampled data

In [0]:
clf = DecisionTreeClassifier(criterion="entropy")
# Train Decision Tree Classifer
clf = clf.fit(X_train_smote,y_train_smote)

In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.23      0.30      0.26     17217
        Okay       0.15      0.21      0.17      8730
        Poor       0.88      0.82      0.85    132815

    accuracy                           0.73    158762
   macro avg       0.42      0.44      0.43    158762
weighted avg       0.77      0.73      0.75    158762

[[  5081   2167   9969]
 [  1951   1815   4964]
 [ 14965   8400 109450]]


Model with Entropy Criterion max_depth set to 15 with Smote Resampled data produced best results

In [0]:
clf = DecisionTreeClassifier(criterion="entropy", max_depth = 15)
# Train Decision Tree Classifer
clf = clf.fit(X_train_smote,y_train_smote)

In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.24      0.44      0.31     17217
        Okay       0.17      0.51      0.25      8730
        Poor       0.92      0.70      0.80    132815

    accuracy                           0.66    158762
   macro avg       0.44      0.55      0.45    158762
weighted avg       0.81      0.66      0.72    158762

[[ 7573  3975  5669]
 [ 2182  4409  2139]
 [21687 17783 93345]]


**On Test Set**

In [0]:
predictions = clf.predict(X_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

        Good       0.24      0.44      0.31     19565
        Okay       0.17      0.49      0.25      9920
        Poor       0.92      0.70      0.80    150926

    accuracy                           0.66    180411
   macro avg       0.44      0.54      0.45    180411
weighted avg       0.81      0.66      0.71    180411

[[  8526   4639   6400]
 [  2433   4884   2603]
 [ 24699  20039 106188]]


---------------------------------------------------------------------------------

### Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier

Base Random Forest Classifier with 100 trees with balanced class weight:

In [0]:
clf = RandomForestClassifier(random_state=42, n_estimators = 100, n_jobs=-1,class_weight="balanced",verbose = 3 )
start = time.time()
clf = clf.fit(X_train,y_train.values.ravel())
end = time.time()
print("Time in seconds : ", end - start)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 100building tree 2 of 100

building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   58.8s


building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  3.4min finished


In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    1.5s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    5.3s finished


              precision    recall  f1-score   support

        Good       0.42      0.14      0.21     17217
        Okay       0.27      0.08      0.12      8730
        Poor       0.86      0.97      0.91    132815

    accuracy                           0.83    158762
   macro avg       0.52      0.40      0.41    158762
weighted avg       0.78      0.83      0.79    158762

[[  2442    617  14158]
 [   883    661   7186]
 [  2492   1160 129163]]


Default 100 trees Random Forest with SMOTE Resampled data

In [0]:
clf = RandomForestClassifier(random_state=42, n_estimators = 100, n_jobs=-1,verbose = 3 )
start = time.time()
clf = clf.fit(X_train_smote,y_train_smote)
end = time.time()
print("Time in seconds : ", end - start)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 2 of 100building tree 3 of 100

building tree 1 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  2.7min


building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  8.3min finished


In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    4.0s finished


              precision    recall  f1-score   support

        Good       0.30      0.31      0.30     17217
        Okay       0.20      0.25      0.22      8730
        Poor       0.89      0.87      0.88    132815

    accuracy                           0.78    158762
   macro avg       0.46      0.48      0.47    158762
weighted avg       0.79      0.78      0.78    158762

[[  5358   2124   9735]
 [  1909   2143   4678]
 [ 10758   6428 115629]]


In [0]:
predictions = clf.predict(X_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    1.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    4.4s finished


              precision    recall  f1-score   support

        Good       0.30      0.31      0.30     19565
        Okay       0.20      0.23      0.21      9920
        Poor       0.89      0.87      0.88    150926

    accuracy                           0.78    180411
   macro avg       0.46      0.47      0.47    180411
weighted avg       0.79      0.78      0.78    180411

[[  6054   2376  11135]
 [  2135   2329   5456]
 [ 12187   7214 131525]]


Random Forest with Gini Index and Max depth 10

In [0]:
clf = RandomForestClassifier(max_depth=10, random_state=42, n_estimators = 100, n_jobs=-1,class_weight="balanced",verbose = 3 )
start = time.time()
clf = clf.fit(X_train,y_train.values.ravel())
end = time.time()
print("Time in seconds : ", end - start)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 100building tree 2 of 100

building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   26.3s


building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.5min finished


In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.3s finished


              precision    recall  f1-score   support

        Good       0.20      0.49      0.28     17217
        Okay       0.16      0.65      0.25      8730
        Poor       0.95      0.57      0.71    132815

    accuracy                           0.56    158762
   macro avg       0.43      0.57      0.41    158762
weighted avg       0.82      0.56      0.64    158762

[[ 8489  5386  3342]
 [ 2282  5646   802]
 [32095 25370 75350]]


Random Forest with Entropy and Max depth 10

In [0]:
clf = RandomForestClassifier(max_depth=10, random_state=42, criterion="entropy", n_estimators = 100, n_jobs=-1,class_weight="balanced",verbose = 3 )
start = time.time()
clf = clf.fit(X_train,y_train.values.ravel())
end = time.time()
print("Time in seconds : ", end - start)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   30.9s


building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.8min finished


In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.3s finished


              precision    recall  f1-score   support

        Good       0.20      0.49      0.28     17217
        Okay       0.15      0.65      0.25      8730
        Poor       0.95      0.57      0.71    132815

    accuracy                           0.56    158762
   macro avg       0.43      0.57      0.41    158762
weighted avg       0.82      0.56      0.64    158762

[[ 8458  5412  3347]
 [ 2292  5640   798]
 [32172 25432 75211]]


Random Forest with Gini Index and Max depth 15

In [0]:
clf = RandomForestClassifier(max_depth=15, random_state=42, n_estimators = 100, n_jobs=-1,class_weight="balanced",verbose = 3 )
start = time.time()
clf = clf.fit(X_train,y_train.values.ravel())
end = time.time()
print("Time in seconds : ", end - start)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 100building tree 2 of 100

building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   38.8s


building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.2min finished


In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.6s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    2.0s finished


              precision    recall  f1-score   support

        Good       0.22      0.50      0.30     17217
        Okay       0.17      0.62      0.27      8730
        Poor       0.94      0.62      0.75    132815

    accuracy                           0.61    158762
   macro avg       0.44      0.58      0.44    158762
weighted avg       0.82      0.61      0.68    158762

[[ 8610  4825  3782]
 [ 2281  5376  1073]
 [28457 21508 82850]]


Random Forest with Entropy and Max depth 15

In [0]:
clf = RandomForestClassifier(max_depth=15, random_state=42, criterion="entropy", n_estimators = 100, n_jobs=-1,class_weight="balanced",verbose = 3 )
start = time.time()
clf = clf.fit(X_train,y_train.values.ravel())
end = time.time()
print("Time in seconds : ", end - start)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 100building tree 2 of 100

building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   41.1s


building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.3min finished


In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.5s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.9s finished


              precision    recall  f1-score   support

        Good       0.22      0.50      0.30     17217
        Okay       0.17      0.61      0.27      8730
        Poor       0.94      0.62      0.75    132815

    accuracy                           0.61    158762
   macro avg       0.44      0.58      0.44    158762
weighted avg       0.82      0.61      0.68    158762

[[ 8582  4837  3798]
 [ 2275  5367  1088]
 [28766 21116 82933]]


Random Forest with Gini Index and Max depth 14

In [0]:
clf = RandomForestClassifier(max_depth=14, random_state=42, n_estimators = 100, n_jobs=-1,class_weight="balanced",verbose = 3 )
start = time.time()
clf = clf.fit(X_train,y_train.values.ravel())
end = time.time()
print("Time in seconds : ", end - start)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   33.9s


building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.0min finished


In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.5s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.8s finished


              precision    recall  f1-score   support

        Good       0.21      0.50      0.30     17217
        Okay       0.17      0.62      0.26      8730
        Poor       0.95      0.61      0.74    132815

    accuracy                           0.60    158762
   macro avg       0.44      0.58      0.44    158762
weighted avg       0.82      0.60      0.67    158762

[[ 8587  4938  3692]
 [ 2297  5420  1013]
 [29372 22106 81337]]


Random Forest with Entropy and Max depth 14

In [0]:
clf = RandomForestClassifier(max_depth=14, random_state=42, criterion="entropy", n_estimators = 100, n_jobs=-1,class_weight="balanced",verbose = 3 )
start = time.time()
clf = clf.fit(X_train,y_train.values.ravel())
end = time.time()
print("Time in seconds : ", end - start)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 100building tree 2 of 100

building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   34.7s


building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.9min finished


In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.5s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.8s finished


              precision    recall  f1-score   support

        Good       0.21      0.49      0.30     17217
        Okay       0.17      0.63      0.26      8730
        Poor       0.95      0.61      0.74    132815

    accuracy                           0.60    158762
   macro avg       0.44      0.58      0.43    158762
weighted avg       0.82      0.60      0.67    158762

[[ 8514  5014  3689]
 [ 2259  5462  1009]
 [29070 22386 81359]]


**Best Results**

Random Forest with Entropy and Max depth 15 with SMOTE Resampling

In [0]:
clf = RandomForestClassifier(max_depth=15, random_state=42, criterion="entropy", n_estimators = 100, n_jobs=-1,class_weight="balanced",verbose = 3 )
start = time.time()
clf = clf.fit(X_train_smote,y_train_smote)
end = time.time()
print("Time in seconds : ", end - start)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 2 of 100building tree 3 of 100
building tree 4 of 100building tree 1 of 100


building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.1min


building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  4.3min finished


In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.0s finished


              precision    recall  f1-score   support

        Good       0.22      0.46      0.30     17217
        Okay       0.17      0.59      0.27      8730
        Poor       0.94      0.66      0.77    132815

    accuracy                           0.63    158762
   macro avg       0.44      0.57      0.45    158762
weighted avg       0.82      0.63      0.69    158762

[[ 7870  4867  4480]
 [ 2055  5186  1489]
 [25682 19630 87503]]


**Prediction on Test Set**

In [0]:
predictions = clf.predict(X_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.2s finished


              precision    recall  f1-score   support

        Good       0.22      0.46      0.30     19565
        Okay       0.17      0.59      0.27      9920
        Poor       0.94      0.66      0.77    150926

    accuracy                           0.63    180411
   macro avg       0.44      0.57      0.45    180411
weighted avg       0.82      0.63      0.69    180411

[[ 8903  5716  4946]
 [ 2378  5843  1699]
 [29157 22365 99404]]


-------------------------------------------------------------------------------

### Gradient Boosting

In [0]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

Base Graient Boosting Classifier

In [0]:
clf = GradientBoostingClassifier( verbose = 3 ,  n_estimators=100,  random_state=10 )
start = time.time()
clf = clf.fit(X_train,y_train.values.ravel())
end = time.time()
print("Time in seconds : ", end - start)

      Iter       Train Loss   Remaining Time 
         1      624925.5044           11.41m
         2      614007.1695           11.28m
         3      605586.8818           11.16m
         4      598897.1672           11.09m
         5      593094.1224           11.05m
         6      588176.3368           10.94m
         7      584395.2487           10.82m
         8      581284.6467           10.70m
         9      578697.2900           10.60m
        10      576452.1915           10.50m
        11      574501.0041           10.41m
        12      572259.6588           10.29m
        13      570643.2931           10.20m
        14      569272.5988           10.11m
        15      567657.1522           10.04m
        16      566542.9565            9.94m
        17      565295.2551            9.83m
        18      564198.4765            9.72m
        19      563196.0635            9.62m
        20      562231.1418            9.52m
        21      561279.9556            9.41m
        2

In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.54      0.00      0.00     17217
        Okay       0.45      0.00      0.01      8730
        Poor       0.84      1.00      0.91    132815

    accuracy                           0.84    158762
   macro avg       0.61      0.34      0.31    158762
weighted avg       0.78      0.84      0.76    158762

[[    29     15  17173]
 [    11     36   8683]
 [    14     29 132772]]


Default Gradient Boost with Respampling :


In [0]:
clf = GradientBoostingClassifier(verbose = 3, n_estimators=100, random_state=10 )
start = time.time()
clf = clf.fit(X_train_smote,y_train_smote)
end = time.time()
print("Time in seconds : ", end - start)

      Iter       Train Loss   Remaining Time 
         1     3137286.7735           59.87m
         2     3077546.4358           59.46m
         3     3028395.8471           58.80m
         4     2986789.5770           58.17m
         5     2952110.5996           57.52m
         6     2921196.5002           56.90m
         7     2895316.7782           56.31m
         8     2872975.5755           55.65m
         9     2853569.0430           55.02m
        10     2836338.5529           54.36m
        11     2821368.0085           53.62m
        12     2807184.1996           52.95m
        13     2794684.9631           52.32m
        14     2783862.8255           51.71m
        15     2773101.1561           51.06m
        16     2764551.1539           50.45m
        17     2756590.2172           49.84m
        18     2748439.3312           49.23m
        19     2740899.2946           48.66m
        20     2734314.7252           48.01m
        21     2728663.7061           47.40m
        2

In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.22      0.45      0.29     17217
        Okay       0.16      0.58      0.25      8730
        Poor       0.93      0.64      0.76    132815

    accuracy                           0.62    158762
   macro avg       0.44      0.56      0.43    158762
weighted avg       0.81      0.62      0.68    158762

[[ 7696  4708  4813]
 [ 2108  5049  1573]
 [25701 21609 85505]]


In [0]:
predictions = clf.predict(X_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

        Good       0.22      0.44      0.29     19565
        Okay       0.16      0.57      0.25      9920
        Poor       0.93      0.64      0.76    150926

    accuracy                           0.62    180411
   macro avg       0.44      0.55      0.43    180411
weighted avg       0.81      0.62      0.68    180411

[[ 8656  5576  5333]
 [ 2490  5642  1788]
 [28969 24614 97343]]


Note : Resampled set did better than non-resampled set

Gradient Boost with Max Depth = 10, Learning Rate = 0.05 with Resampling:




In [0]:
clf = GradientBoostingClassifier( verbose = 3 ,learning_rate = 0.05,  n_estimators=100, max_depth=10, max_features='sqrt',n_iter_no_change=5, tol=80, subsample=0.8, random_state=10 )
start = time.time()
clf = clf.fit(X_train_smote,y_train_smote)
end = time.time()
print("Time in seconds : ", end - start)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1     2280282.1650        7693.5820           28.99m
         2     2249405.8880        7667.4634           29.62m
         3     2219902.7628        7357.2452           29.87m
         4     2194398.7741        6340.7127           29.93m
         5     2168318.6449        6510.2760           29.63m
         6     2144769.1892        5845.3809           29.33m
         7     2122037.4973        5632.1168           28.86m
         8     2106025.3612        4036.6482           28.55m
         9     2088044.2600        4388.8881           28.24m
        10     2071582.4653        4152.9763           27.77m
        11     2057102.6658        3548.9257           27.41m
        12     2041841.6641        3733.0058           27.09m
        13     2027475.1911        3654.5097           26.72m
        14     2015297.5375        2980.8359           26.39m
        15     2003422.6082        3001.5170           26.11m
       

In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.24      0.46      0.31     17217
        Okay       0.18      0.56      0.27      8730
        Poor       0.93      0.69      0.79    132815

    accuracy                           0.66    158762
   macro avg       0.45      0.57      0.46    158762
weighted avg       0.81      0.66      0.71    158762

[[ 7872  4270  5075]
 [ 2098  4855  1777]
 [23450 17884 91481]]


Gradient Boost with Max Depth = 12, Learning Rate = 0.1 with Resampling:

In [0]:
clf = GradientBoostingClassifier( verbose = 3 ,learning_rate = 0.1,  n_estimators=100, max_depth=12, max_features='sqrt', subsample=0.8,n_iter_no_change=5, tol=10000, random_state=10 )
start = time.time()
clf = clf.fit(X_train_smote,y_train_smote)
end = time.time()
print("Time in seconds : ", end - start)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1     2238687.8195       17949.9446           41.29m
         2     2178392.6988       14908.2484           43.37m
         3     2122772.3306       13748.0594           44.68m
         4     2076931.5996       11285.8357           45.59m
         5     2038992.9869        9443.0068           45.06m
         6     2004338.5790        8526.1501           44.91m
         7     1974514.5773        7305.3305           44.88m
         8     1950295.2084        5999.3856           44.72m
         9     1927686.6427        5459.3719           44.40m
        10     1907947.6653        4916.4758           43.99m
        11     1888941.1895        4511.8652           43.46m
        12     1868905.9785        4779.3364           43.21m
        13     1853450.8895        3922.1110           42.83m
        14     1839848.5538        3235.3558           42.43m
        15     1825155.3696        3592.5602           42.04m
       

In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.22      0.46      0.30     17217
        Okay       0.17      0.59      0.27      8730
        Poor       0.94      0.66      0.77    132815

    accuracy                           0.63    158762
   macro avg       0.44      0.57      0.45    158762
weighted avg       0.82      0.63      0.69    158762

[[ 7983  4772  4462]
 [ 2072  5165  1493]
 [25521 20146 87148]]


Gradient Boost with Max Depth = 15, Learning Rate = 0.05 with Resampling:

In [0]:
clf = GradientBoostingClassifier( verbose = 3 ,learning_rate = 0.05,  n_estimators=100, max_depth=15, max_features='sqrt', subsample=0.8,n_iter_no_change=5, tol=10000, random_state=10 )
start = time.time()
clf = clf.fit(X_train_smote,y_train_smote)
end = time.time()
print("Time in seconds : ", end - start)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1     2269230.9284       10264.0122           75.57m
         2     2228026.6646       10101.5885           93.06m
         3     2189447.1710        9432.2649          101.16m
         4     2155877.0819        8214.7129          102.77m
         5     2123772.0610        7838.1687          101.93m
         6     2092924.7387        7532.5909          103.09m
         7     2064029.9055        7032.7827          102.85m
         8     2041344.9180        5564.7476          101.15m
         9     2018116.1587        5584.8410          100.21m
        10     1994663.4192        5726.5608           99.91m
        11     1973208.7749        5127.9105           98.57m
        12     1954734.5278        4395.2367           97.24m
        13     1934940.1800        4887.8650           97.11m
        14     1918162.2110        3993.4596           95.87m
        15     1901823.1695        3931.2787           95.04m
       

In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.23      0.47      0.31     17217
        Okay       0.18      0.58      0.27      8730
        Poor       0.93      0.67      0.78    132815

    accuracy                           0.64    158762
   macro avg       0.45      0.57      0.45    158762
weighted avg       0.82      0.64      0.70    158762

[[ 8029  4560  4628]
 [ 2074  5059  1597]
 [24871 18764 89180]]


Gradient Boost with Max Depth = 15, Learning Rate = 0.1 with Resampling:

In [0]:
clf = GradientBoostingClassifier( verbose = 3 ,learning_rate = 0.1,  n_estimators=100, max_depth=15, max_features='sqrt', subsample=0.8,n_iter_no_change=5, tol=10000, random_state=10 )
start = time.time()
clf = clf.fit(X_train_smote,y_train_smote)
end = time.time()
print("Time in seconds : ", end - start)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1     2229229.6033       20028.0040           72.88m
         2     2156501.8057       17772.1193           89.63m
         3     2092285.5646       15626.7394          100.09m
         4     2039157.4550       12905.3994          102.84m
         5     1992986.0183       11186.8274          102.32m
         6     1949913.9806       10411.5568          102.77m
         7     1913241.9969        8773.3780          102.60m
         8     1883754.0229        7165.0544          101.07m
         9     1854957.9870        6789.7723          100.34m
        10     1827919.7937        6464.4315          100.11m
        11     1804289.4358        5431.3689           99.24m
        12     1782971.0418        4980.0940           97.98m
        13     1761782.9788        5124.5454           97.53m
        14     1743723.3029        4163.8290           96.09m
        15     1727410.9167        3757.2450           95.23m
       

In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.24      0.46      0.32     17217
        Okay       0.18      0.55      0.28      8730
        Poor       0.93      0.70      0.80    132815

    accuracy                           0.66    158762
   macro avg       0.45      0.57      0.46    158762
weighted avg       0.82      0.66      0.72    158762

[[ 7916  4292  5009]
 [ 2108  4836  1786]
 [22963 17253 92599]]


With the test set 

In [0]:
predictions = clf.predict(X_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

        Good       0.24      0.46      0.32     19565
        Okay       0.18      0.54      0.27      9920
        Poor       0.93      0.70      0.80    150926

    accuracy                           0.66    180411
   macro avg       0.45      0.57      0.46    180411
weighted avg       0.82      0.66      0.72    180411

[[  8991   5080   5494]
 [  2392   5404   2124]
 [ 25940  19653 105333]]


 Gradient Boost with Max Depth = 15, Learning Rate = 0.1 with Resampling and Reducing tol to 7000

In [0]:
clf = GradientBoostingClassifier( verbose = 3 ,learning_rate = 0.1,  n_estimators=100, max_depth=15, max_features='sqrt', subsample=0.8,n_iter_no_change=5, tol=7000, random_state=10 )
start = time.time()
clf = clf.fit(X_train_smote,y_train_smote)
end = time.time()
print("Time in seconds : ", end - start)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1     2229229.6033       20028.0040           74.71m
         2     2156501.8057       17772.1193           92.38m
         3     2092285.5646       15626.7394          102.73m
         4     2039157.4550       12905.3994          104.72m
         5     1992986.0183       11186.8274          105.23m
         6     1949913.9806       10411.5568          106.32m
         7     1913241.9969        8773.3780          105.96m
         8     1883754.0229        7165.0544          104.53m
         9     1854957.9870        6789.7723          103.92m
        10     1827919.7937        6464.4315          103.81m
        11     1804289.4358        5431.3689          103.12m
        12     1782971.0418        4980.0940          102.01m
        13     1761782.9788        5124.5454          101.92m
        14     1743723.3029        4163.8290          100.57m
        15     1727410.9167        3757.2450          100.29m
       

In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.24      0.46      0.32     17217
        Okay       0.18      0.54      0.27      8730
        Poor       0.93      0.70      0.80    132815

    accuracy                           0.67    158762
   macro avg       0.45      0.57      0.46    158762
weighted avg       0.81      0.67      0.72    158762

[[ 7916  4178  5123]
 [ 2140  4715  1875]
 [22682 16722 93411]]


Gradient Boost with Max Depth = 15, Learning Rate = 0.1 with Resampling and Reducing tol to 5000

In [0]:
clf = GradientBoostingClassifier( verbose = 3 ,learning_rate = 0.1,  n_estimators=100, max_depth=15, max_features='sqrt', subsample=0.8,n_iter_no_change=5, tol=5000, random_state=10 )
start = time.time()
clf = clf.fit(X_train_smote,y_train_smote)
end = time.time()
print("Time in seconds : ", end - start)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1     2229229.6033       20028.0040           74.18m
         2     2156501.8057       17772.1193           89.91m
         3     2092285.5646       15626.7394           99.69m
         4     2039157.4550       12905.3994          101.35m
         5     1992986.0183       11186.8274          100.66m
         6     1949913.9806       10411.5568          101.09m
         7     1913241.9969        8773.3780          100.60m
         8     1883754.0229        7165.0544           98.94m
         9     1854957.9870        6789.7723           97.88m
        10     1827919.7937        6464.4315           97.37m
        11     1804289.4358        5431.3689           96.25m
        12     1782971.0418        4980.0940           94.92m
        13     1761782.9788        5124.5454           94.33m
        14     1743723.3029        4163.8290           92.80m
        15     1727410.9167        3757.2450           91.86m
       

In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.26      0.45      0.33     17217
        Okay       0.19      0.49      0.28      8730
        Poor       0.92      0.74      0.82    132815

    accuracy                           0.70    158762
   macro avg       0.46      0.56      0.48    158762
weighted avg       0.81      0.70      0.74    158762

[[ 7740  3605  5872]
 [ 2140  4247  2343]
 [20088 13952 98775]]


In [0]:
predictions = clf.predict(X_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

        Good       0.26      0.45      0.33     19565
        Okay       0.19      0.47      0.27      9920
        Poor       0.92      0.75      0.83    150926

    accuracy                           0.70    180411
   macro avg       0.46      0.56      0.48    180411
weighted avg       0.81      0.70      0.74    180411

[[  8803   4276   6486]
 [  2449   4685   2786]
 [ 22695  15615 112616]]


In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.26      0.45      0.33     17217
        Okay       0.19      0.49      0.28      8730
        Poor       0.92      0.74      0.82    132815

    accuracy                           0.70    158762
   macro avg       0.46      0.56      0.48    158762
weighted avg       0.81      0.70      0.74    158762

[[ 7740  3605  5872]
 [ 2140  4247  2343]
 [20088 13952 98775]]


Reducing tol to 3000  and no of iterations to 3:

In [0]:
clf2 = GradientBoostingClassifier( verbose = 3 ,learning_rate = 0.1,  n_estimators=100, max_depth=15, max_features='sqrt', subsample=0.8,n_iter_no_change=3, tol=3000, random_state=10 )
start = time.time()
clf2 = clf2.fit(X_train_smote,y_train_smote)
end = time.time()
print("Time in seconds : ", end - start)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1     2229229.6033       20028.0040           65.06m
         2     2156501.8057       17772.1193           80.53m
         3     2092285.5646       15626.7394           89.72m
         4     2039157.4550       12905.3994           91.37m
         5     1992986.0183       11186.8274           90.68m
         6     1949913.9806       10411.5568           91.16m
         7     1913241.9969        8773.3780           91.15m
         8     1883754.0229        7165.0544           90.09m
         9     1854957.9870        6789.7723           89.59m
        10     1827919.7937        6464.4315           89.52m
        11     1804289.4358        5431.3689           88.98m
        12     1782971.0418        4980.0940           88.00m
        13     1761782.9788        5124.5454           87.80m
        14     1743723.3029        4163.8290           87.04m
        15     1727410.9167        3757.2450           86.81m
       

In [0]:
predictions = clf2.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.26      0.45      0.33     17217
        Okay       0.19      0.49      0.28      8730
        Poor       0.92      0.74      0.82    132815

    accuracy                           0.70    158762
   macro avg       0.46      0.56      0.48    158762
weighted avg       0.81      0.70      0.74    158762

[[ 7740  3605  5872]
 [ 2140  4247  2343]
 [20088 13952 98775]]


In [0]:
predictions = clf2.predict(X_train)
print(classification_report(y_train,predictions))
print(confusion_matrix(y_train,predictions))

              precision    recall  f1-score   support

        Good       0.29      0.51      0.37    126262
        Okay       0.23      0.57      0.33     64016
        Poor       0.93      0.75      0.83    973973

    accuracy                           0.72   1164251
   macro avg       0.49      0.61      0.51   1164251
weighted avg       0.82      0.72      0.76   1164251

[[ 64421  23526  38315]
 [ 13261  36339  14416]
 [142036  97967 733970]]


Previous model did better

Best Model was Gradient Boost with Max Depth = 15, Learning Rate = 0.1 with Resampling and Reducing tol to 5000

**Predictions on testset**

In [0]:
predictions = clf.predict(X_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

        Good       0.26      0.45      0.33     19565
        Okay       0.19      0.47      0.27      9920
        Poor       0.92      0.75      0.83    150926

    accuracy                           0.70    180411
   macro avg       0.46      0.56      0.48    180411
weighted avg       0.81      0.70      0.74    180411

[[  8803   4276   6486]
 [  2449   4685   2786]
 [ 22695  15615 112616]]


----------------------------------------------------------------------

### Cat Boost Classifier

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.12, random_state = 0, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.12, random_state = 0, stratify=y_train)

In [0]:
sc = StandardScaler()
X_train.loc[:,["price","description_len","title_len","param_combined_len","image_top_1"]] = sc.fit_transform(X_train[["price","description_len","title_len","param_combined_len","image_top_1"]])
X_test.loc[:,["price","description_len","title_len","param_combined_len","image_top_1"]] = sc.transform(X_test[["price","description_len","title_len","param_combined_len","image_top_1"]])
X_valid.loc[:,["price","description_len","title_len","param_combined_len","image_top_1"]] = sc.transform(X_valid[["price","description_len","title_len","param_combined_len","image_top_1"]])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [0]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [0]:
pip install ipywidgets

In [0]:
pip install catboost

     |████████████████████████████████| 63.9MB 45kB/s 


In [0]:
from catboost import CatBoostClassifier
from  sklearn.utils.class_weight import compute_class_weight

In [0]:
arr_y = np.unique(y.values.ravel())
sample_weights_data = compute_class_weight(class_weight = "balanced", classes = arr_y, y = y.values.ravel())
sample_weights_data

array([3.07365701, 6.06224244, 0.39845413])

In [0]:
cate_features_index = np.array([0,1,2])

In [0]:
cate_features_index

array([0, 1, 2])

Using MultiClass Loss function for the dataset

Using all edfault parameters including default 1000 trees

In [0]:
clf = CatBoostClassifier(random_seed=42,loss_function='MultiClass', class_weights=sample_weights_data)
clf.fit(X_train, y_train, cat_features=cate_features_index )

0:	learn: 1.0904747	total: 1.52s	remaining: 25m 15s
1:	learn: 1.0828443	total: 2.9s	remaining: 24m 6s
2:	learn: 1.0757271	total: 4.28s	remaining: 23m 41s
3:	learn: 1.0689030	total: 5.54s	remaining: 22m 58s
4:	learn: 1.0624592	total: 6.67s	remaining: 22m 6s
5:	learn: 1.0564419	total: 7.92s	remaining: 21m 52s
6:	learn: 1.0508201	total: 9.21s	remaining: 21m 46s
7:	learn: 1.0454109	total: 10.3s	remaining: 21m 21s
8:	learn: 1.0402768	total: 11.4s	remaining: 20m 56s
9:	learn: 1.0354693	total: 12.5s	remaining: 20m 41s
10:	learn: 1.0309135	total: 13.7s	remaining: 20m 30s
11:	learn: 1.0266235	total: 14.8s	remaining: 20m 18s
12:	learn: 1.0225063	total: 16s	remaining: 20m 17s
13:	learn: 1.0186259	total: 17.4s	remaining: 20m 22s
14:	learn: 1.0148554	total: 18.4s	remaining: 20m 10s
15:	learn: 1.0113419	total: 19.7s	remaining: 20m 11s
16:	learn: 1.0080443	total: 20.8s	remaining: 20m 2s
17:	learn: 1.0047871	total: 22s	remaining: 20m
18:	learn: 1.0017339	total: 23.2s	remaining: 19m 59s
19:	learn: 0.99

In [0]:
y_pred = clf.predict(X_valid)
print(classification_report(y_valid,y_pred))
print(confusion_matrix(y_valid,y_pred))

              precision    recall  f1-score   support

        Good       0.23      0.51      0.31     17217
        Okay       0.17      0.62      0.26      8730
        Poor       0.95      0.62      0.75    132815

    accuracy                           0.61    158762
   macro avg       0.45      0.58      0.44    158762
weighted avg       0.82      0.61      0.68    158762

[[ 8703  4759  3755]
 [ 2248  5416  1066]
 [27267 22629 82919]]


Cat Boost with MultiClass Loss function, AUC as metric and iterations = 60, max_depth = 15, learning_rate = 0.1

In [0]:
clf = CatBoostClassifier(random_seed=42,loss_function='MultiClass',eval_metric = 'AUC',class_weights=sample_weights_data,early_stopping_rounds=2,iterations=60, max_depth=15, learning_rate=0.1)
clf.fit(X_train, y_train, cat_features=cate_features_index )

0:	total: 28.4s	remaining: 27m 53s
1:	total: 58.5s	remaining: 28m 16s
2:	total: 1m 31s	remaining: 28m 58s
3:	total: 2m 6s	remaining: 29m 24s
4:	total: 2m 41s	remaining: 29m 41s
5:	total: 3m 18s	remaining: 29m 47s
6:	total: 3m 56s	remaining: 29m 50s
7:	total: 4m 35s	remaining: 29m 48s
8:	total: 5m 14s	remaining: 29m 43s
9:	total: 5m 55s	remaining: 29m 35s
10:	total: 6m 35s	remaining: 29m 23s
11:	total: 7m 17s	remaining: 29m 9s
12:	total: 7m 58s	remaining: 28m 51s
13:	total: 8m 41s	remaining: 28m 32s
14:	total: 9m 23s	remaining: 28m 9s
15:	total: 10m 6s	remaining: 27m 47s
16:	total: 10m 50s	remaining: 27m 24s
17:	total: 11m 33s	remaining: 26m 58s
18:	total: 12m 17s	remaining: 26m 31s
19:	total: 13m 1s	remaining: 26m 2s
20:	total: 13m 44s	remaining: 25m 31s
21:	total: 14m 28s	remaining: 25m
22:	total: 14m 29s	remaining: 23m 18s
23:	total: 15m 14s	remaining: 22m 51s
24:	total: 15m 59s	remaining: 22m 23s
25:	total: 16m 44s	remaining: 21m 53s
26:	total: 17m 28s	remaining: 21m 22s
27:	total: 

In [0]:
y_pred = clf.predict(X_valid)
print(classification_report(y_valid,y_pred))
print(confusion_matrix(y_valid,y_pred))

              precision    recall  f1-score   support

        Good       0.22      0.50      0.31     17217
        Okay       0.16      0.61      0.26      8730
        Poor       0.94      0.62      0.75    132815

    accuracy                           0.61    158762
   macro avg       0.44      0.58      0.44    158762
weighted avg       0.82      0.61      0.68    158762

[[ 8587  4763  3867]
 [ 2322  5312  1096]
 [27914 22200 82701]]


Cat Boost with MultiClass Loss function, AUC as metric and iterations = 60, max_depth = 15, learning_rate = 0.2

In [0]:
clf = CatBoostClassifier(random_seed=42,loss_function='MultiClass',eval_metric = 'AUC',class_weights=sample_weights_data,early_stopping_rounds=2,iterations=60, max_depth=15, learning_rate=0.2)
clf.fit(X_train, y_train, cat_features=cate_features_index )

0:	total: 47s	remaining: 46m 10s
1:	total: 1m 33s	remaining: 45m 1s
2:	total: 2m 24s	remaining: 45m 51s
3:	total: 3m 11s	remaining: 44m 43s
4:	total: 3m 57s	remaining: 43m 34s
5:	total: 4m 44s	remaining: 42m 43s
6:	total: 5m 30s	remaining: 41m 43s
7:	total: 6m 18s	remaining: 41m
8:	total: 7m 6s	remaining: 40m 14s
9:	total: 7m 52s	remaining: 39m 22s
10:	total: 8m 38s	remaining: 38m 30s
11:	total: 9m 24s	remaining: 37m 37s
12:	total: 10m 10s	remaining: 36m 47s
13:	total: 10m 57s	remaining: 35m 59s
14:	total: 11m 44s	remaining: 35m 13s
15:	total: 12m 30s	remaining: 34m 22s
16:	total: 13m 16s	remaining: 33m 34s
17:	total: 14m 3s	remaining: 32m 49s
18:	total: 14m 49s	remaining: 32m
19:	total: 15m 36s	remaining: 31m 13s
20:	total: 16m 22s	remaining: 30m 24s
21:	total: 17m 9s	remaining: 29m 38s
22:	total: 17m 55s	remaining: 28m 50s
23:	total: 18m 43s	remaining: 28m 4s
24:	total: 19m 28s	remaining: 27m 16s
25:	total: 20m 14s	remaining: 26m 28s
26:	total: 20m 59s	remaining: 25m 39s
27:	total: 2

In [0]:
y_pred = clf.predict(X_valid)
print(classification_report(y_valid,y_pred))
print(confusion_matrix(y_valid,y_pred))

              precision    recall  f1-score   support

        Good       0.23      0.51      0.31     17217
        Okay       0.16      0.59      0.26      8730
        Poor       0.94      0.63      0.76    132815

    accuracy                           0.62    158762
   macro avg       0.45      0.58      0.44    158762
weighted avg       0.82      0.62      0.68    158762

[[ 8743  4569  3905]
 [ 2402  5154  1174]
 [27164 21632 84019]]


Cat Boost with MultiClass Loss function, AUC as metric and iterations=100, depth=15, learning_rate=0.1

In [0]:
clf = CatBoostClassifier(random_seed=42,loss_function='MultiClass',eval_metric = 'AUC',class_weights=sample_weights_data,early_stopping_rounds=2,iterations=100, depth=15, learning_rate=0.1)
clf.fit(X_train, y_train, cat_features=cate_features_index )

0:	total: 22.9s	remaining: 37m 42s
1:	total: 45.1s	remaining: 36m 50s
2:	total: 1m 9s	remaining: 37m 13s
3:	total: 1m 36s	remaining: 38m 31s
4:	total: 2m 3s	remaining: 39m 12s
5:	total: 2m 31s	remaining: 39m 32s
6:	total: 3m	remaining: 40m 4s
7:	total: 3m 31s	remaining: 40m 28s
8:	total: 4m 1s	remaining: 40m 45s
9:	total: 4m 33s	remaining: 40m 58s
10:	total: 5m 4s	remaining: 41m 5s
11:	total: 5m 36s	remaining: 41m 9s
12:	total: 6m 9s	remaining: 41m 14s
13:	total: 6m 42s	remaining: 41m 9s
14:	total: 7m 15s	remaining: 41m 9s
15:	total: 7m 48s	remaining: 40m 59s
16:	total: 8m 21s	remaining: 40m 46s
17:	total: 8m 53s	remaining: 40m 32s
18:	total: 9m 27s	remaining: 40m 18s
19:	total: 10m 1s	remaining: 40m 4s
20:	total: 10m 35s	remaining: 39m 48s
21:	total: 11m 9s	remaining: 39m 32s
22:	total: 11m 9s	remaining: 37m 22s
23:	total: 11m 42s	remaining: 37m 5s
24:	total: 12m 17s	remaining: 36m 51s
25:	total: 12m 51s	remaining: 36m 34s
26:	total: 13m 25s	remaining: 36m 17s
27:	total: 13m 58s	remai

In [0]:
y_pred = clf.predict(X_valid)
print(classification_report(y_valid,y_pred))
print(confusion_matrix(y_valid,y_pred))

              precision    recall  f1-score   support

        Good       0.23      0.51      0.31     17217
        Okay       0.17      0.60      0.26      8730
        Poor       0.94      0.63      0.76    132815

    accuracy                           0.62    158762
   macro avg       0.45      0.58      0.44    158762
weighted avg       0.82      0.62      0.68    158762

[[ 8712  4607  3898]
 [ 2335  5238  1157]
 [27088 21597 84130]]


The above model was the best one

**Preictions on test set**

In [0]:
y_pred = clf.predict(X_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

        Good       0.23      0.51      0.32     19565
        Okay       0.17      0.60      0.26      9920
        Poor       0.94      0.64      0.76    150926

    accuracy                           0.62    180411
   macro avg       0.45      0.58      0.44    180411
weighted avg       0.82      0.62      0.68    180411

[[ 9887  5427  4251]
 [ 2634  5914  1372]
 [30606 24419 95901]]
